In [3]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

In [4]:
pd.set_option("display.max_columns", 500)

In [27]:
#Listing currently existing tables in the database
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
        SELECT name
        FROM sqlite_master 
        WHERE type ='table' 
        AND name NOT LIKE 'sqlite_%';
        """ 

    test_df = pd.read_sql(query, db)

display(test_df)

,name
0,taxonomy
1,hop_teaming
2,veronica
3,stan
4,courtney


## Test Veronica

In [18]:
#All NPIs not in Veronica's
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    WITH veronica_npi AS (
        SELECT npi
        FROM veronica
    )
    SELECT *
    FROM npidata
    WHERE npi NOT IN veronica_npi
    """

    missing_veronica = pd.read_sql(query, db)

display(missing_veronica)
missing_veronica.to_csv("missing_veronica.csv")

,npi,entity_type_code,provider_org_name,provider_last_name,provider_first_name,provider_middle_name,provider_name_prefix,provider_name_suffix,provider_credential,provider_business_address_1,provider_business_address_2,provider_business_city,provider_business_state,provider_business_zip,taxonomy_code,provider_business_zip5
0,1720081532,1,None,SMART,KELLY,M,None,None,P.A,141 HILLCREST DR,None,CLARKSVILLE,TN,37043,363A00000X,37043
1,1013910892,1,None,LEJEUNE,EMIKO,JANE,None,None,M.S.,1740 MEMORIAL DR,None,CLARKSVILLE,TN,370434561,231H00000X,37043
2,1942204805,1,None,DAVIS,WILLIAM,GLENN,None,None,M.D.,220 N CHANCERY ST,None,MC MINNVILLE,TN,371102551,207W00000X,37110
3,1972507317,1,None,STEPHENS,LOPASTEEDA,DEROSA,None,None,None,915 N CHANCERY ST,STE 120,MCMINNVILLE,TN,371101575,152W00000X,37110
4,1912902396,1,None,JAMIESON,MARY JANE,None,None,None,None,1589 SPARTA ST,STE 305,MC MINNVILLE,TN,371101389,207Q00000X,37110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2724,1659961381,1,None,RHINE,MEAGAN,ALEXANDRIA,None,None,None,2197 MADISON ST STE 106,None,CLARKSVILLE,TN,370435253,225100000X,37043
2725,1649860305,2,"VANDERBILT BEDFORD HOSPITAL, LLC",None,None,None,None,None,None,2835 HIGHWAY 231 N,None,SHELBYVILLE,TN,371607327,207RI0011X,37160
2726,1881285294,1,None,AUSTIN,LADELLA,None,None,None,LCSW,5736 MANCHESTER HWY,None,MORRISON,TN,373577503,1041C0700X,37357
2727,1093306599,1,None,PRAETE,DIANA,None,None,None,LCSW,775 WEATHERLY DR STE F,None,CLARKSVILLE,TN,370438915,1041C0700X,37043


In [12]:
#Group by City
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    WITH veronica_npi AS (
        SELECT npi
        FROM veronica
    )
    SELECT provider_business_city, COUNT(provider_business_city) AS count
    FROM npidata
    WHERE npi NOT IN veronica_npi
    GROUP BY provider_business_city
    """

    test_df = pd.read_sql(query, db)

display(test_df)

,provider_business_city,count
0,ADAMS,10
1,ALEXANDRIA,7
2,BEACHGROVE,1
3,BEECHGROVE,1
4,BELL BUCKLE,15
5,BON AQUA,6
6,CLARKSVILLE,1270
7,LEWISBURG,242
8,LIBERTY,2
9,LYNNVILLE,6


In [13]:
#All Veronica's NPIs not in ours
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    WITH parker_npi AS (
        SELECT npi
        FROM npidata
    )
    SELECT *
    FROM veronica
    WHERE npi NOT IN parker_npi
    """

    test_df = pd.read_sql(query, db)

display(test_df)

,npi,entity_type_code,provider_organization_name,provider_last_name,provider_first_name,provider_middle_name,provider_name_prefix,provider_name_suffix,provider_credential,address_01,address_02,city,state,zip_9,zip_5,primary_taxonomy,Classification
0,1598756330,2.0,"TOWN OF GREENLAND, NH",None,None,None,None,None,None,575 PORTSMOUTH AVE,None,GREENLAND,NH,38402251.0,38402,341600000X,Ambulance
1,1811971641,2.0,TOWN OF NEW DURHAM,None,None,None,None,None,None,4 MAIN ST,None,NEW DURHAM,NH,38552201.0,38552,341600000X,Ambulance
2,1306822580,1.0,None,FODERO,SALLY,WOODIE,DR.,None,None,655 PORTSMOUTH AVE,None,GREENLAND,NH,38402264.0,38402,231H00000X,Audiologist
3,1316910342,1.0,None,MEYER,ANNETTE,SORIANO,None,None,NP,1009 WINDCROSS COURT,None,FRANKLIN,SC,37067.0,37067,363LA2200X,Nurse Practitioner
4,1326082074,1.0,None,RAMOS,JOSE,ALBERTO,DR.,None,DMD,1 BAYSIDE RD,None,GREENLAND,NH,38402117.0,38402,1223G0001X,Dentist
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,1104461151,1.0,None,STEVENS,MARISA,CROMMETT,None,None,None,559 PORTSMOUTH AVE,None,GREENLAND,NH,38402251.0,38402,363LW0102X,Nurse Practitioner
72,1639703614,1.0,None,HUTCHINSON,CHERYL,LYNN,None,None,ARNP,9 SPRING HILL RD,None,GREENLAND,NH,38402600.0,38402,363LF0000X,Nurse Practitioner
73,1265035133,1.0,None,DANG,CHRISTINA,PHAM,None,None,PHARMD,1450 GREENLAND RD,None,GREENLAND,NH,38402438.0,38402,183500000X,Pharmacist
74,1144824806,1.0,None,PENNEY,BARBARA,None,None,None,PHARMD,1450 GREENLAND RD,None,GREENLAND,NH,38402438.0,38402,183500000X,Pharmacist


## End Veronica

## Test Stan

In [15]:
#All NPIs not in Stan's
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    SELECT *
    FROM stan
    LIMIT 1
    """

    test_df = pd.read_sql(query, db)

display(test_df)

,Unnamed: 0,from_npi,to_npi,patient_count,transaction_count,average_day_wait,Phys_NPI,Phys_Office,Phys_Last_Name,Phys_First_Name,Phys_Class,Phys_Spec,Hosp_NPI,Hospital,Hosp_Class,Hosp_Spec
0,0,1003819046,1023055126,11,11,13.182,1003819046,None,NYLANDER,BARBARA,Obstetrics & Gynecology,Gynecology,1023055126,"HCA HEALTH SERVICES OF TENNESSEE, INC.",General Acute Care Hospital,None


In [23]:
#All hops not in Stan's
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    WITH stan_from AS (
        SELECT from_npi
        FROM stan
    ), stan_to AS (
        SELECT to_npi
        FROM stan
    )
    SELECT *
    FROM filtered_hop_teaming
    WHERE from_npi NOT IN stan_from
    UNION
    SELECT *
    FROM filtered_hop_teaming
    WHERE to_npi NOT IN stan_to
    """

    missing_hops = pd.read_sql(query, db)

display(missing_hops)
missing_hops.to_csv('missing_hops.csv')

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,1003013160,1093741464,65,90,23.578,30.333
1,1003013160,1104202761,28,50,35.540,41.353
2,1003013160,1437194669,58,91,33.066,44.959
3,1003013160,1497700231,45,60,42.350,56.368
4,1003013160,1497828321,27,54,33.185,36.578
...,...,...,...,...,...,...
37251,1992985949,1700878360,31,95,9.989,21.369
37252,1992985949,1811955917,29,57,23.351,36.616
37253,1992985949,1861478489,178,296,22.649,46.893
37254,1992985949,1972577351,39,53,20.585,37.518


In [24]:
#All Stan's hops not in mine
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    WITH parker_from AS (
        SELECT from_npi
        FROM filtered_hop_teaming
    ), parker_to AS (
        SELECT to_npi
        FROM filtered_hop_teaming
    )
    SELECT *
    FROM stan
    WHERE from_npi NOT IN parker_from
    UNION
    SELECT *
    FROM stan
    WHERE to_npi NOT IN parker_to
    """

    parker_missing = pd.read_sql(query, db)

display(parker_missing)

,Unnamed: 0,from_npi,to_npi,patient_count,transaction_count,average_day_wait,Phys_NPI,Phys_Office,Phys_Last_Name,Phys_First_Name,Phys_Class,Phys_Spec,Hosp_NPI,Hospital,Hosp_Class,Hosp_Spec
0,0,1003819046,1023055126,11,11,13.182,1003819046,None,NYLANDER,BARBARA,Obstetrics & Gynecology,Gynecology,1023055126,"HCA HEALTH SERVICES OF TENNESSEE, INC.",General Acute Care Hospital,None
1,30,1437152485,1023055126,12,13,98.231,1437152485,None,MORGAN,LISA,Obstetrics & Gynecology,None,1023055126,"HCA HEALTH SERVICES OF TENNESSEE, INC.",General Acute Care Hospital,None
2,106,1275536468,1285644237,11,11,85.545,1275536468,None,ASAD,MUHAMMAD,Internal Medicine,Cardiovascular Disease,1285644237,"SUMNER REGIONAL HEALTH SYSTEMS, INC",General Acute Care Hospital,None
3,165,1194728394,1023055126,16,19,8.316,1194728394,None,ANDERSON,ANNE,Obstetrics & Gynecology,None,1023055126,"HCA HEALTH SERVICES OF TENNESSEE, INC.",General Acute Care Hospital,None
4,447,1316940893,1396882205,25,25,60.360,1316940893,None,LUNDY,JEFFREY,Family Medicine,None,1396882205,VANDERBILT UNIVERSITY MEDICAL CENTER,General Acute Care Hospital,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
807,35722,1376065623,1306889597,24,25,0.120,1376065623,None,CURTIS,AMANDA,Nurse Practitioner,Family,1306889597,VANDERBILT UNIVERSITY MEDICAL CENTER,General Acute Care Hospital,None
808,35725,1780109967,1396882205,12,12,5.583,1780109967,None,ENSTROM,JEANNE,Nurse Practitioner,Acute Care,1396882205,VANDERBILT UNIVERSITY MEDICAL CENTER,General Acute Care Hospital,None
809,35726,1558887091,1164590386,17,17,7.294,1558887091,None,DUNCAN,JEFFREY,"Nurse Anesthetist, Certified Registered",None,1164590386,SAINT THOMAS RUTHERFORD HOSPITAL,General Acute Care Hospital,None
810,35728,1306362637,1780778969,21,21,1.000,1306362637,None,WILGING,ERIN,"Nurse Anesthetist, Certified Registered",None,1780778969,SAINT THOMAS WEST HOSPITAL,General Acute Care Hospital,None


In [26]:
#All NPIs not in Stan's
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    SELECT COUNT(*)
    FROM filtered_hop_teaming
    """

    test_df = pd.read_sql(query, db)

display(test_df)

,COUNT(*)
0,44072


## Test Stan

## Test Courtney

In [21]:
#All hops not in Courtney's
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    WITH courtney_from AS (
        SELECT from_npi
        FROM courtney
    ), courtney_to AS (
        SELECT to_npi
        FROM courtney
    )
    SELECT *
    FROM filtered_hop_teaming
    WHERE from_npi NOT IN courtney_from
    UNION
    SELECT *
    FROM filtered_hop_teaming
    WHERE to_npi NOT IN courtney_to
    """

    test_df = pd.read_sql(query, db)

display(test_df)

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,1003013160,1093741464,65,90,23.578,30.333
1,1003013160,1104202761,28,50,35.540,41.353
2,1003013160,1437194669,58,91,33.066,44.959
3,1003013160,1497700231,45,60,42.350,56.368
4,1003013160,1497828321,27,54,33.185,36.578
...,...,...,...,...,...,...
37251,1992985949,1700878360,31,95,9.989,21.369
37252,1992985949,1811955917,29,57,23.351,36.616
37253,1992985949,1861478489,178,296,22.649,46.893
37254,1992985949,1972577351,39,53,20.585,37.518


## End Courtney

## Test Mine 

In [25]:
#All NPIs not in Stan's
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    SELECT *
    FROM npidata
    WHERE npi = '1003013160'
    """

    test_df = pd.read_sql(query, db)

display(test_df)

,npi,entity_type_code,provider_org_name,provider_last_name,provider_first_name,provider_middle_name,provider_name_prefix,provider_name_suffix,provider_credential,provider_business_address_1,provider_business_address_2,provider_business_city,provider_business_state,provider_business_zip,taxonomy_code,provider_business_zip5
0,1003013160,1,None,GRABENSTEIN,WILLIAM,P.,DR.,None,M.D.,1822 MEMORIAL DR,None,CLARKSVILLE,TN,370434605,207Q00000X,37043


In [6]:
# Test number of distinct Zip Codes
# We differ from other groups, because we already joined to npidata. There must be some zips with no npi.
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    SELECT COUNT(DISTINCT provider_business_zip5) AS zips
    FROM npidata;
    """ 
    
    test = pd.read_sql(query, db)

test

,zips
0,127


In [7]:
# Check number of npis We have more than other groups - not sure why.
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    SELECT COUNT(*)
    FROM npidata
    """ 
    
    test = pd.read_sql(query, db)

test

,COUNT(*)
0,41067


In [8]:
# Check number of npis per entity type. We have more than other groups - not sure why.
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    SELECT COUNT(npi)
    FROM npidata
    WHERE entity_type_code = 1
    """ 
    
    test = pd.read_sql(query, db)

test

,COUNT(npi)
0,32789


In [9]:
# Check number of to/from npis and pairs. We have more than other groups - following suite of npidata.
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    SELECT COUNT(*)
    FROM filtered_hop_teaming
    """ 
    
    test = pd.read_sql(query, db)

test

,COUNT(*)
0,44072


In [10]:
# Check number of to/from npis and pairs. We have more than other groups - following suite of npidata.
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    SELECT COUNT(DISTINCT to_npi)
    FROM filtered_hop_teaming
    """ 
    
    test = pd.read_sql(query, db)

test

,COUNT(DISTINCT to_npi)
0,1363


In [ ]:
# Check number of to/from npis and pairs. We have more than other groups - following suite of npidata.
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    SELECT COUNT(DISTINCT from_npi)
    FROM filtered_hop_teaming
    """ 
    
    test = pd.read_sql(query, db)

test

In [ ]:
# Check number of npis per entity type. We have more than other groups - not sure why.
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    SELECT *
    FROM npidata
    LIMIT 1
    """ 
    
    test = pd.read_sql(query, db)

test

In [ ]:
# Find NPIs with largest number of referrals
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    SELECT SUM(transaction_count) AS total_referrals, to_npi, n.provider_org_name
    FROM filtered_hop_teaming AS f
    JOIN npidata AS n
    ON f.to_npi = n.npi
    GROUP BY to_npi
    ORDER BY total_referrals DESC
    LIMIT 20;
    """ 
    
    test = pd.read_sql(query, db)

test

In [ ]:
# Find competitor hosptitals with the largest number of total referrals.
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    SELECT SUM(transaction_count) AS total_referrals, to_npi, n.provider_org_name, t.classification
    FROM filtered_hop_teaming AS f
    JOIN npidata AS n
    ON f.to_npi = n.npi
    JOIN taxonomy AS t
    USING(taxonomy_code)
    WHERE classification = 'Family Medicine'
    GROUP BY to_npi, provider_org_name
    --HAVING provider_org_name LIKE '%VUMC%'
    ORDER BY total_referrals DESC
    LIMIT 50;
    """ 
    
    test = pd.read_sql(query, db)

test

In [ ]:
# Find competitor hosptitals with the largest number of total referrals.
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    SELECT classification, SUM(transaction_count)
    FROM filtered_hop_teaming AS f
    JOIN npidata AS n
    ON f.to_npi = n.npi
    JOIN taxonomy AS t
    USING(taxonomy_code)
    --WHERE provider_org_name LIKE '%VANDERBILT%'
    GROUP BY classification
    ORDER BY SUM(transaction_count) DESC;
    """ 
    
    test = pd.read_sql(query, db)

test

In [ ]:
# Find addresses of all Vanderbilt locations
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    SELECT *
    FROM npidata
    WHERE provider_org_name LIKE '%VANDERBILT%'
    GROUP BY provider_business_address_1
        , provider_business_city
        , provider_business_state
        , provider_business_zip5
    """ 
    
    test = pd.read_sql(query, db)

test

In [ ]:
# Find competitor hosptitals with the largest number of total referrals.
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    WITH vandy_address AS (
        SELECT *
        FROM npidata
        WHERE provider_org_name LIKE '%VANDERBILT%'
        GROUP BY provider_business_address_1
            , provider_business_city
            , provider_business_state
            , provider_business_zip5
    )
    SELECT *
    FROM npidata
    --JOIN vandy_address AS va USING(npi)
    WHERE provider_org_name NOT LIKE '%VANDERBILT%'
    AND provider_business_address_1 IN (SELECT provider_business_address_1 FROM vandy_address)
    AND provider_business_city IN (SELECT provider_business_city FROM vandy_address)
    AND provider_business_zip5 IN (SELECT provider_business_zip5 FROM vandy_address)
    """ 
    
    test = pd.read_sql(query, db)

test

In [ ]:
# TESTING SQUARE
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    SELECT provider_business_state, COUNT(provider_business_state)
    FROM npidata
    GROUP BY provider_business_state
    """ 
    
    test = pd.read_sql(query, db)

test

In [ ]:
# TESTING SQUARE
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    SELECT *
    FROM npidata
    LIMIT 1;
    """ 
    
    test = pd.read_sql(query, db)

test

In [ ]:
# Tingting's code for exporting to Neo4j

nodes = list(set(ht.from_npi.tolist() + ht.to_npi.tolist()))
node_df = pd.DataFrame({'npi:ID': nodes})
node_df[':LABEL'] = "Provider"
node_df.to_csv('import/nodes.csv', index = False)
edges = pd.DataFrame({':START_ID' : ht.from_npi, 'patient_count': ht.patient_count, 
                      'transaction_count': ht.transaction_count, 
                     ':END_ID' : ht.to_npi})
edges[':TYPE'] = 'REFERRED_TO'
edges.to_csv('import/edges.csv', index = False)

In [ ]:
# Test to confirm the tables loaded.
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    SELECT *
    FROM taxonomy
    LIMIT 1;
    """ 
    
    test = pd.read_sql(query, db)

test

In [ ]:
# Test to confirm the tables loaded.
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    SELECT DISTINCT classification
    FROM taxonomy
    """ 
    
    test = pd.read_sql(query, db)

test

In [ ]:
# Test to confirm the tables loaded.
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    SELECT npidata.*
    FROM npidata
    JOIN taxonomy USING (taxonomy_code)
    WHERE classification = 'train'
    """ 
    
    test = pd.read_sql(query, db)

test